<a href="https://colab.research.google.com/github/ykitaguchi77/GravCont_classification_colab/blob/master/Sort_Olympia_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Sort Oplympia dataset**

In [1]:
#GDriveをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
parent_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset"

%cd $parent_path

/content/drive/MyDrive/Deep_learning/Olympia_dataset


In [ ]:
import glob
import os
import shutil
import cv2


treated_id = os.listdir("treated_500px")
untreated_id = os.listdir("untreated")

# **横幅640pixelにresizeして別フォルダに保存**

In [ ]:
# Create destination directories
basename_lists = [treated_id, untreated_id]
dst_dirs = ["treated_640px", "untreated_640px"]
for dir in dst_dirs:
    if os.path.exists(dir):
        shutil.rmtree(dir)
    os.makedirs(dir)

In [ ]:
def resize_by_width(orig_dir, dst_dir, basename_list, width):
    path_list = glob.glob(f"{orig_dir}/*")
    for path in path_list:
        if os.path.basename(path) in basename_list:
            # Load the image
            img = cv2.imread(path)
            # Get the aspect ratio of the original image
            aspect_ratio = float(img.shape[0]) / img.shape[1]
            # Calculate the new height of the image
            new_height = int(640 * aspect_ratio)
            # Resize the image to 640xnew_height while maintaining aspect ratio
            resized_img = cv2.resize(img, (640, new_height), interpolation = cv2.INTER_LINEAR)
            # Save the resized image
            cv2.imwrite(f"{dst_dir}/{os.path.basename(path)}", resized_img)
            print(path)

In [ ]:
for dst_dir, basename_list in zip(dst_dirs, basename_lists):
    resize_by_width("dataset", dir, basename_list, width=640)
    print(f"{dst_dir} done!!")

# **CSVファイルから抽出**

In [41]:
import pandas as pd
import os

# CSVファイルのパスを指定して、Pandasのデータフレームに読み込む
csv_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/AI_modified_20220809.csv"
df = pd.read_csv(csv_path, encoding="cp932")

# データフレームから"分類"が"コントロール"の行の"写真No."をリストに変換する
control_list = df.loc[df["分類"] == "コントロール"]["写真No."].tolist()

# "treated_500px"フォルダ内のファイル名をリストに変換する
# ファイル名から拡張子を除いた部分を取得する
treated_list = [int(os.path.basename(path).split(".")[0]) for path in os.listdir("/content/drive/MyDrive/Deep_learning/Olympia_dataset/treated_500px")]


In [39]:
import random

def choose_items(lst, n, random_seed=1):
    """
    リストからランダムにn個の要素を選択する関数。

    Args:
        lst (list): リスト
        n (int): 選択する要素数
        random_seed (int): 乱数のシード (デフォルト: 1)

    Returns:
        list: 選択された要素のリスト
    """
    random.seed(random_seed)
    return random.sample(lst, n)

#treated_listの数をcontrol_listと揃える (=169枚)
print(f"control_list: {len(control_list)}")

treated_list = choose_items(treated_list, len(control_list))
print(f"treated_list: {len(treated_list)}")


control_list: 169
treated_list: 169


In [42]:
import glob
import os
import shutil
from tqdm import tqdm
import cv2

image_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset"
dst_dir_control = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/control_640px"
dst_dir_treated = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/treated_640px"



#横幅を640pixelに調整
def resize_by_width(orig_dir, dst_dir, basename_list, width):
    path_list = glob.glob(f"{orig_dir}/*")
    for path in tqdm(path_list, desc='Resize images'):
        if os.path.basename(path) in basename_list:
            # Load the image
            img = cv2.imread(path)
            # Get the aspect ratio of the original image
            aspect_ratio = float(img.shape[0]) / img.shape[1]
            # Calculate the new height of the image
            new_height = int(640 * aspect_ratio)
            # Resize the image to 640xnew_height while maintaining aspect ratio
            resized_img = cv2.resize(img, (640, new_height), interpolation = cv2.INTER_LINEAR)
            # Save the resized image
            cv2.imwrite(f"{dst_dir}/{os.path.basename(path)}", resized_img)
            # Print the path of the processed image
            #print(path)

dst_dirs = [dst_dir_control, dst_dir_treated]
image_lists = [control_list, treated_list]

for dst_dir, image_list in zip(dst_dirs, image_lists):
    if os.path.exists(dst_dir):
        shutil.rmtree(dst_dir)
    os.makedirs(dst_dir)

    for image in tqdm(glob.glob(f"{image_dir}/*"), desc=f'Extract and copy images'):
        if int(os.path.basename(image).split(".")[0]) in image_list:
            shutil.copy(image, dst_dir)

    # Call the resize function with the destination directory and its contents
    resize_by_width(dst_dir, dst_dir, glob.glob(f"dst_dir/*"), 640)

Resize images: 100%|██████████| 561/561 [00:00<00:00, 297584.99it/s]
